In [ ]:
!pip install lightgbm

In [1]:
import gc
import os
import pprint as pp

import lightgbm as lgb
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
order_id = np.load('prepped_data/order_id.npy')
product_id = np.load('prepped_data/product_id.npy')
features = np.load('prepped_data/features.npy')
feature_names = np.load('prepped_data/feature_names.npy', allow_pickle=True)
label = np.load('prepped_data/label.npy')

In [ ]:
product_df = pd.DataFrame(data=features, columns=feature_names)
product_df['order_id'] = order_id
product_df['product_id'] = product_id
product_df['label'] = label


In [ ]:
del order_id, product_id, features, feature_names, label
gc.collect()

In [ ]:
drop_cols = [i for i in product_df.columns if i.startswith('dmf')]
drop_cols += ['order_id', 'product_id', 'label']

# training
train_df = product_df[product_df['label'] != -1]
test_df = product_df[product_df['label'] == -1]
train_df, val_df = train_test_split(train_df, train_size=.99)


In [ ]:
del product_df
gc.collect()

In [ ]:
Y_train, Y_val = train_df['label'].astype(int).astype(float), val_df['label'].astype(int).astype(float)
X_train, X_val = train_df.drop(drop_cols, axis=1), val_df.drop(drop_cols, axis=1)

In [ ]:
del train_df
gc.collect()

In [ ]:
test_orders = test_df['order_id']
test_products = test_df['product_id']
test_labels = test_df['label']
X_test = test_df.drop(drop_cols, axis=1)

In [ ]:
del test_df
gc.collect()

In [ ]:
params = {
    'task': 'train',
    'boosting_type': 'gbdt',
    'objective': 'binary',
    'metric': {'binary_logloss'},
    'learning_rate': .02,
    'num_leaves': 32,
    'max_depth': 12,
    'feature_fraction': 0.35,
    'bagging_fraction': 0.9,
    'bagging_freq': 2,
}
rounds = 15000
d_train = lgb.Dataset(X_train, label=Y_train)
d_valid = lgb.Dataset(X_val, label=Y_val)

In [ ]:
del X_train, X_val, Y_train, Y_val


In [ ]:
def save_model_callback(env):
    if (env.iteration + 1) % 50 == 0:
        env.model.save_model(f"model_iter_{env.iteration + 1}.txt")

In [ ]:
valid_sets = [d_train, d_valid]
valid_names = ['train', 'valid']
gbdt = lgb.train(params,
                 d_train,
                 rounds,
                 valid_sets=valid_sets,
                 valid_names=valid_names,
                 callbacks=[lgb.log_evaluation(period=20)])

In [ ]:
features = gbdt.feature_name_
importance = list(gbdt.feature_importance())
importance = zip(features, importance)
importance = sorted(importance, key=lambda x: x[1])
total = sum(j for i, j in importance)
importance = [(i, float(j)/total) for i, j in importance]
pp.pprint(importance)

In [ ]:
test_preds = gbdt.predict(X_test, num_iteration=gbdt.best_iteration)


In [ ]:
dirname = 'predictions_gbm'
if not os.path.isdir(dirname):
    os.makedirs(dirname)

np.save(os.path.join(dirname, 'order_ids.npy'), test_orders)
np.save(os.path.join(dirname, 'product_ids.npy'), test_products)
np.save(os.path.join(dirname, 'predictions.npy'), test_preds)
np.save(os.path.join(dirname, 'labels.npy'), test_labels)